In [1]:
from datetime import datetime, timedelta
import requests
import re
from io import BytesIO
import pdfplumber
#import wget
import pandas as pd

In [2]:
URL = 'https://www.aerocivil.gov.co/servicios-a-la-navegacion/servicio-de-informacion-aeronautica-ais/Documents/Charlie1.pdf'
raw = requests.get(URL).content
pdf = pdfplumber.open(BytesIO(raw))


In [3]:
#pdf=pdfplumber.open("Charlie1.pdf")

In [4]:
texto=[]
for paginas in pdf.pages:
    texto.extend(paginas.extract_text().splitlines()[:-1])
    

In [5]:
busqueda='NR TEXTO' 

In [6]:
def substringindex(l, s):
    for i in range(len(l)):
        if l[i].find(s)!=-1:
            return i
    return None # Or -1

In [7]:
index=substringindex(texto,busqueda)

In [8]:
texto1=texto[index+1:]
    

In [11]:
notamlist=[]
x=[]
for i in texto1:
    
    if (i[0]=='C' or i[0]=='D') and i[1]==' ':
        notamlist.append(x)
        x=[]
        x.append(i)
    else:
        x.append(i)
notamlist.append(x)

In [12]:
notamlist.pop(0)
#notamlist

[]

In [13]:
len(notamlist)
    

312

In [14]:
a=[]

for j in notamlist:
    b=[]
    for i in j:
        b.append(i.replace('(cid:13)',''))
    a.append(b)
    

In [15]:
#for i in a:
#    for j in range(len(i)):
#       print(i[j])
#    print('---------')
    

In [16]:
notamslist =[]
for i in a:
    sigla=i[0].find('(S')
    if sigla!=-1:
        linea1=i[0].split()
        linea2=i[1].split()
        serie=linea1.pop(0)
        numero=linea1.pop(0)
        ano=linea1.pop(1)
        tipo='AERODROMO'
        lugar=(i[0][sigla:]).strip("()")
        inicio=datetime.strptime(linea2.pop(0),'%y%m%d%H%M')
        fin=linea2.pop(1)
        if len(fin) == 10:
            fin=datetime.strptime(fin,'%y%m%d%H%M')
        elif fin == 'PERM':
            fin=datetime.now() + timedelta(days=365)
        lentexto=len(i)
        texto=''
        for j in range(1,lentexto):
            texto=texto+i[j]+' '
        fila={'serie': serie,'numero':numero,'ano':ano,'tipo':tipo,'lugar':lugar,'inicio':inicio,'fin':fin,'texto':texto}
        
    else:
        linea1=i[0].split()
        linea2=i[1].split()
        serie=linea1.pop(0)
        numero=linea1.pop(0)
        ano=linea1.pop(1)
        tipo='FIR/UIR'
        lugar=linea1.pop()
        inicio=datetime.strptime(linea2.pop(0),'%y%m%d%H%M')
        fin=linea2.pop(1)
        if len(fin) == 10:
            fin=datetime.strptime(fin,'%y%m%d%H%M')
        elif fin == 'PERM':
            fin=datetime.now() + timedelta(days=365)
        lentexto=len(i)
        texto=''
        for j in range(2,lentexto):
            texto=texto+i[j]+' '
        fila={'serie': serie,'numero':numero,'ano':ano,'tipo':tipo,'lugar':lugar,'inicio':inicio,'fin':fin,'texto':texto}
        
    notamslist.append(fila)        
    
    #print(i[0])
    #print(i[0].split())
 

In [17]:
df = pd.DataFrame(notamslist)

In [18]:
df

,serie,numero,ano,tipo,lugar,inicio,fin,texto
0,C,3305,20,AERODROMO,SKBS,2020-10-12 04:53:00,2023-08-01 18:45:31.704483,"2010120453 / PERM , RWY 18/36 CAMBIA WID A 1..."
1,C,3306,20,AERODROMO,SKBS,2020-10-12 04:57:00,2023-08-01 18:45:31.704533,"2010120457 / PERM , FRANJA RWY 18/36 CAMBIA ..."
2,C,4065,20,AERODROMO,SKVV,2020-11-29 14:22:00,2023-08-01 18:45:31.704551,"2011291422 / PERM , LGT BORDE APN DE VIRAJE ..."
3,C,4066,20,FIR/UIR,BOGOTA,2020-11-29 15:22:00,2023-08-01 18:45:31.704564,BANCO DATOS OPMET U/S RPLC NOTAM C1861 /17
4,C,4076,20,AERODROMO,SKPS,2020-11-29 16:45:00,2023-08-01 18:45:31.704576,"2011291645 / PERM , RWY 02/20 CHG PCN A 48/F..."
...,...,...,...,...,...,...,...,...
307,C,2515,22,FIR/UIR,BOGOTA,2022-08-01 01:12:00,2022-08-31 23:59:00.000000,CHECKLIST NOTAM SERIE C VIGENTES A LAS 0112Z D...
308,C,2516,22,FIR/UIR,BOGOTA,2022-08-01 11:35:00,2022-11-01 23:59:00.000000,SSR CAR U/S RPLC NOTAM C2434 /22
309,C,2518,22,AERODROMO,SKBG,2022-08-01 18:37:00,2022-11-02 23:59:00.000000,"2208011837 / 2211022359 , AD, SE ESTA CORTAN..."
310,C,2519,22,AERODROMO,SKTL,2022-08-04 13:00:00,2022-09-04 21:00:00.000000,"2208041300 / 2209042100 DLY BTN 1300-2100, Z..."


In [21]:
#html = df.to_html()
html=df1.to_html()
text_file = open("index.html", "w")
text_file.write(html)
text_file.close()

In [44]:
df1=df.groupby("lugar").count()

In [48]:
df1=df.groupby('lugar').size().reset_index(name = 'obs').sort_values(['obs'],ascending=True )

In [50]:
df1=df.sort_values(['lugar'])

In [20]:
aerodromos=['SKBO','SKRG','SKCL','SKPE','SKAR','SKMR','SKCG','SKBQ','SKSM','SKSP']
df1=df[df['lugar'].isin(aerodromos)]
df1=df1.sort_values(['lugar','inicio'],ascending=[True,False])